# Import Python libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from imblearn.over_sampling import SMOTENC
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load data

In [2]:
# load ASHRAE global dataset
data = pd.read_csv('data/ashrae_global_database.csv')
data.head()

/var/folders/yd/7n_986614z1d2f6f174fvgm80000gn/T/ipykernel_13981/2801882121.py:2: DtypeWarning: Columns (5,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/ashrae_global_database.csv')


,index,record_id,building_id,timestamp,season,subject_id,age,gender,ht,wt,ta,ta_h,ta_m,ta_l,top,tr,tg,tg_h,tg_m,tg_l,rh,vel,vel_h,vel_m,vel_l,met,clo,activity_10,activity_20,activity_30,activity_60,thermal_sensation,thermal_acceptability,thermal_preference,thermal_comfort,air_movement_acceptability,air_movement_preference,blind_curtain,fan,window,door,heater,t_out,rh_out,t_out_monthly,t_out_isd,rh_out_isd,set,pmv,ppd,pmv_ce,ppd_ce,t_mot_isd
0,0,1,1,1995-05-18T00:00:00Z,winter,10.0,NaN,female,NaN,NaN,22.3,NaN,22.3,NaN,22.625000,22.950000,NaN,NaN,NaN,NaN,61.0,0.03,NaN,0.03,NaN,1.706485,0.95,NaN,NaN,NaN,NaN,2.0,acceptable,cooler,4.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,28.1,0.50,10.2,0.38,8.0,17.4
1,1,2,1,1995-05-18T00:00:00Z,winter,14.0,NaN,female,NaN,NaN,23.0,NaN,23.0,NaN,23.567143,24.134286,NaN,NaN,NaN,NaN,59.0,0.08,NaN,0.08,NaN,1.109215,1.07,NaN,NaN,NaN,NaN,2.0,unacceptable,cooler,2.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,27.2,0.40,8.4,0.40,8.4,17.4
2,2,3,1,1995-05-18T00:00:00Z,winter,15.0,NaN,female,NaN,NaN,22.0,NaN,22.0,NaN,22.045714,22.091429,NaN,NaN,NaN,NaN,61.0,0.04,NaN,0.04,NaN,1.211604,0.88,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,24.8,-0.07,5.1,-0.07,5.1,17.4
3,3,4,1,1995-05-18T00:00:00Z,winter,17.0,NaN,female,NaN,NaN,23.2,NaN,23.2,NaN,23.765152,24.330303,NaN,NaN,NaN,NaN,58.0,0.03,NaN,0.03,NaN,1.774744,0.59,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,NaN,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,25.5,0.31,7.0,0.14,5.4,17.4
4,4,5,1,1995-05-18T00:00:00Z,winter,21.0,NaN,male,NaN,NaN,22.2,NaN,22.2,NaN,22.638571,23.077143,NaN,NaN,NaN,NaN,62.0,0.15,NaN,0.15,NaN,1.484642,0.83,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,26.0,0.05,5.0,-0.06,5.1,17.4


In [3]:
# load ASHRAE meta data
meta_data = pd.read_csv('data/ashrae_db_metadata.csv')
meta_data.head()

,building_id,building_id_inf,contributor,publication,region,country,city,lat,lon,climate,building_type,cooling_type,year,records,has_age,has_ec,has_timestamp,timezone,met_source,isd_station,isd_distance,database,quality_assurance
0,1,no,David Rowe,NaN,oceania,australia,sydney,-33.868820,151.209295,humid subtropical,office,mixed mode,1995,170,no,no,yes,Australia/Sydney,rp884,947680-99999,2.3,1.0,pass
1,2,no,David Rowe,NaN,oceania,australia,sydney,-33.868820,151.209295,humid subtropical,office,air conditioned,1996,83,no,no,yes,Australia/Sydney,rp884,947680-99999,2.3,1.0,pass
2,3,no,Fred Baumann,NaN,americas,usa,grand rapids,42.963360,-85.668086,humid midlatitude,office,air conditioned,1992,85,no,no,yes,America/Detroit,rp884,726350-94860,14.9,1.0,pass
3,4,no,David Rowe,NaN,oceania,australia,sydney,-33.868820,151.209295,humid subtropical,office,mixed mode,1995,137,no,no,yes,Australia/Sydney,NaN,947680-99999,2.3,1.0,pass
4,5,no,Gail Brager,"Benton, C. C. & Brager, G. S. Advanced Custome...",americas,usa,auburn,32.609857,-85.480783,mediterranean,office,air conditioned,1993,128,no,no,yes,America/Chicago,rp884,722288-99999,2.1,1.0,pass


In [4]:
# merge ASHRAE data with meta data on cooling type information
data = pd.merge(data, meta_data[['building_id','cooling_type']], 
                on='building_id', 
                how='left', 
                suffixes=['', ''])
data.head()

,index,record_id,building_id,timestamp,season,subject_id,age,gender,ht,wt,ta,ta_h,ta_m,ta_l,top,tr,tg,tg_h,tg_m,tg_l,rh,vel,vel_h,vel_m,vel_l,met,clo,activity_10,activity_20,activity_30,activity_60,thermal_sensation,thermal_acceptability,thermal_preference,thermal_comfort,air_movement_acceptability,air_movement_preference,blind_curtain,fan,window,door,heater,t_out,rh_out,t_out_monthly,t_out_isd,rh_out_isd,set,pmv,ppd,pmv_ce,ppd_ce,t_mot_isd,cooling_type
0,0,1,1,1995-05-18T00:00:00Z,winter,10.0,NaN,female,NaN,NaN,22.3,NaN,22.3,NaN,22.625000,22.950000,NaN,NaN,NaN,NaN,61.0,0.03,NaN,0.03,NaN,1.706485,0.95,NaN,NaN,NaN,NaN,2.0,acceptable,cooler,4.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,28.1,0.50,10.2,0.38,8.0,17.4,mixed mode
1,1,2,1,1995-05-18T00:00:00Z,winter,14.0,NaN,female,NaN,NaN,23.0,NaN,23.0,NaN,23.567143,24.134286,NaN,NaN,NaN,NaN,59.0,0.08,NaN,0.08,NaN,1.109215,1.07,NaN,NaN,NaN,NaN,2.0,unacceptable,cooler,2.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,27.2,0.40,8.4,0.40,8.4,17.4,mixed mode
2,2,3,1,1995-05-18T00:00:00Z,winter,15.0,NaN,female,NaN,NaN,22.0,NaN,22.0,NaN,22.045714,22.091429,NaN,NaN,NaN,NaN,61.0,0.04,NaN,0.04,NaN,1.211604,0.88,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,24.8,-0.07,5.1,-0.07,5.1,17.4,mixed mode
3,3,4,1,1995-05-18T00:00:00Z,winter,17.0,NaN,female,NaN,NaN,23.2,NaN,23.2,NaN,23.765152,24.330303,NaN,NaN,NaN,NaN,58.0,0.03,NaN,0.03,NaN,1.774744,0.59,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,NaN,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,25.5,0.31,7.0,0.14,5.4,17.4,mixed mode
4,4,5,1,1995-05-18T00:00:00Z,winter,21.0,NaN,male,NaN,NaN,22.2,NaN,22.2,NaN,22.638571,23.077143,NaN,NaN,NaN,NaN,62.0,0.15,NaN,0.15,NaN,1.484642,0.83,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,26.0,0.05,5.0,-0.06,5.1,17.4,mixed mode


# Perform column relabelling

In [5]:
# rename columns to human readable format
name_mapping = {'cooling_type':'Mode',
                'ta':'Indoor Temp',
                'ta_h':'Indoor Temp 1.1m',
                'ta_m':'Indoor Temp 0.6m',
                'ta_l':'Indoor Temp 0.1m',
                'rh':'Indoor Humidity',
                'vel':'Air Velocity',
                'vel_h':'Air Velocity 1.1m',
                'vel_m':'Air Velocity 0.6m',
                'vel_l':'Air Velocity 0.1m',
                'tg':'Globe Temperature',
                'tg_h':'Globe Temperature 1.1m',
                'tg_m':'Globe Temperature 0.6m',
                'tg_l':'Globe Temperature 0.1m',
                't_out':'Outdoor Temp',
                'rh_out':'Outdoor Humidity',
                'thermal_preference':'Thermal Preference',
                'thermal_acceptability':'Thermal Acceptability',
                'air_movement_preference':'Air Movement Preference'}

data.rename(columns=name_mapping, inplace=True)
data.head()

,index,record_id,building_id,timestamp,season,subject_id,age,gender,ht,wt,Indoor Temp,Indoor Temp 1.1m,Indoor Temp 0.6m,Indoor Temp 0.1m,top,tr,Globe Temperature,Globe Temperature 1.1m,Globe Temperature 0.6m,Globe Temperature 0.1m,Indoor Humidity,Air Velocity,Air Velocity 1.1m,Air Velocity 0.6m,Air Velocity 0.1m,met,clo,activity_10,activity_20,activity_30,activity_60,thermal_sensation,Thermal Acceptability,Thermal Preference,thermal_comfort,air_movement_acceptability,Air Movement Preference,blind_curtain,fan,window,door,heater,Outdoor Temp,Outdoor Humidity,t_out_monthly,t_out_isd,rh_out_isd,set,pmv,ppd,pmv_ce,ppd_ce,t_mot_isd,Mode
0,0,1,1,1995-05-18T00:00:00Z,winter,10.0,NaN,female,NaN,NaN,22.3,NaN,22.3,NaN,22.625000,22.950000,NaN,NaN,NaN,NaN,61.0,0.03,NaN,0.03,NaN,1.706485,0.95,NaN,NaN,NaN,NaN,2.0,acceptable,cooler,4.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,28.1,0.50,10.2,0.38,8.0,17.4,mixed mode
1,1,2,1,1995-05-18T00:00:00Z,winter,14.0,NaN,female,NaN,NaN,23.0,NaN,23.0,NaN,23.567143,24.134286,NaN,NaN,NaN,NaN,59.0,0.08,NaN,0.08,NaN,1.109215,1.07,NaN,NaN,NaN,NaN,2.0,unacceptable,cooler,2.0,unacceptable,more,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,27.2,0.40,8.4,0.40,8.4,17.4,mixed mode
2,2,3,1,1995-05-18T00:00:00Z,winter,15.0,NaN,female,NaN,NaN,22.0,NaN,22.0,NaN,22.045714,22.091429,NaN,NaN,NaN,NaN,61.0,0.04,NaN,0.04,NaN,1.211604,0.88,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,24.8,-0.07,5.1,-0.07,5.1,17.4,mixed mode
3,3,4,1,1995-05-18T00:00:00Z,winter,17.0,NaN,female,NaN,NaN,23.2,NaN,23.2,NaN,23.765152,24.330303,NaN,NaN,NaN,NaN,58.0,0.03,NaN,0.03,NaN,1.774744,0.59,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,NaN,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,25.5,0.31,7.0,0.14,5.4,17.4,mixed mode
4,4,5,1,1995-05-18T00:00:00Z,winter,21.0,NaN,male,NaN,NaN,22.2,NaN,22.2,NaN,22.638571,23.077143,NaN,NaN,NaN,NaN,62.0,0.15,NaN,0.15,NaN,1.484642,0.83,NaN,NaN,NaN,NaN,0.0,acceptable,no change,5.0,acceptable,no change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,26.0,0.05,5.0,-0.06,5.1,17.4,mixed mode


# Perform data cleaning

In [6]:
# fill missing values for certain temperature and air velocity sensors by using similar sensors deployed

def fill_missing_with_average(data, target_col, substitute_cols):
    """
    Fill missing data in the specified column(s) with the average of non-null data from other columns.

    Parameters:
        data (pd.DataFrame): The input DataFrame.
        target_col (str): Column name of the feature of interest
        substitute_cols (list of str): Columns containing data from similar sensors 

    Returns:
        pd.DataFrame: A DataFrame with missing values in the specified columns filled with the calculated average.
    """
    # Copy the input DataFrame to avoid modifying the original
    temp_data = data.copy()

    # Create a mask for missing values in the target column
    missing_mask = temp_data[target_col].isnull()

    # Check if all columns in the row are NaN
    all_columns_nan = temp_data[[target_col] + substitute_cols].isnull().all(axis=1)

    # Calculate the average from other columns excluding the target column
    avg_values = temp_data.loc[~(missing_mask & all_columns_nan), substitute_cols].mean(axis=1)

    # Fill missing values with the calculated average
    temp_data.loc[missing_mask, target_col] = avg_values

    return temp_data

data = fill_missing_with_average(data, 'Indoor Temp', ['Indoor Temp 1.1m','Indoor Temp 0.6m','Indoor Temp 0.1m'])
data = fill_missing_with_average(data, 'Air Velocity', ['Air Velocity 1.1m','Air Velocity 0.6m','Air Velocity 0.1m'])
data = fill_missing_with_average(data, 'Globe Temperature', ['Globe Temperature 1.1m','Globe Temperature 0.6m','Globe Temperature 0.1m'])

In [7]:
# Standardise preference labels
data['Thermal Acceptability'] = data['Thermal Acceptability'].replace('acceptable', 'Acceptable')
data['Thermal Acceptability'] = data['Thermal Acceptability'].replace('unacceptable', 'Unacceptable')

data['Thermal Preference'] = data['Thermal Preference'].replace('cooler', 'Cooler')
data['Thermal Preference'] = data['Thermal Preference'].replace('no change', 'No Change')
data['Thermal Preference'] = data['Thermal Preference'].replace('warmer', 'Warmer')

data['Air Movement Preference'] = data['Air Movement Preference'].replace('more', 'More')
data['Air Movement Preference'] = data['Air Movement Preference'].replace('no change', 'No Change')
data['Air Movement Preference'] = data['Air Movement Preference'].replace('less', 'Less')

data['Mode'] = data['Mode'].replace('air conditioned', 'AC')
data['Mode'] = data['Mode'].replace('naturally ventilated', 'NV')

In [8]:
data.head()

,index,record_id,building_id,timestamp,season,subject_id,age,gender,ht,wt,Indoor Temp,Indoor Temp 1.1m,Indoor Temp 0.6m,Indoor Temp 0.1m,top,tr,Globe Temperature,Globe Temperature 1.1m,Globe Temperature 0.6m,Globe Temperature 0.1m,Indoor Humidity,Air Velocity,Air Velocity 1.1m,Air Velocity 0.6m,Air Velocity 0.1m,met,clo,activity_10,activity_20,activity_30,activity_60,thermal_sensation,Thermal Acceptability,Thermal Preference,thermal_comfort,air_movement_acceptability,Air Movement Preference,blind_curtain,fan,window,door,heater,Outdoor Temp,Outdoor Humidity,t_out_monthly,t_out_isd,rh_out_isd,set,pmv,ppd,pmv_ce,ppd_ce,t_mot_isd,Mode
0,0,1,1,1995-05-18T00:00:00Z,winter,10.0,NaN,female,NaN,NaN,22.3,NaN,22.3,NaN,22.625000,22.950000,NaN,NaN,NaN,NaN,61.0,0.03,NaN,0.03,NaN,1.706485,0.95,NaN,NaN,NaN,NaN,2.0,Acceptable,Cooler,4.0,unacceptable,More,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,28.1,0.50,10.2,0.38,8.0,17.4,mixed mode
1,1,2,1,1995-05-18T00:00:00Z,winter,14.0,NaN,female,NaN,NaN,23.0,NaN,23.0,NaN,23.567143,24.134286,NaN,NaN,NaN,NaN,59.0,0.08,NaN,0.08,NaN,1.109215,1.07,NaN,NaN,NaN,NaN,2.0,Unacceptable,Cooler,2.0,unacceptable,More,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,27.2,0.40,8.4,0.40,8.4,17.4,mixed mode
2,2,3,1,1995-05-18T00:00:00Z,winter,15.0,NaN,female,NaN,NaN,22.0,NaN,22.0,NaN,22.045714,22.091429,NaN,NaN,NaN,NaN,61.0,0.04,NaN,0.04,NaN,1.211604,0.88,NaN,NaN,NaN,NaN,0.0,Acceptable,No Change,5.0,acceptable,No Change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,24.8,-0.07,5.1,-0.07,5.1,17.4,mixed mode
3,3,4,1,1995-05-18T00:00:00Z,winter,17.0,NaN,female,NaN,NaN,23.2,NaN,23.2,NaN,23.765152,24.330303,NaN,NaN,NaN,NaN,58.0,0.03,NaN,0.03,NaN,1.774744,0.59,NaN,NaN,NaN,NaN,0.0,Acceptable,No Change,5.0,acceptable,NaN,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,25.5,0.31,7.0,0.14,5.4,17.4,mixed mode
4,4,5,1,1995-05-18T00:00:00Z,winter,21.0,NaN,male,NaN,NaN,22.2,NaN,22.2,NaN,22.638571,23.077143,NaN,NaN,NaN,NaN,62.0,0.15,NaN,0.15,NaN,1.484642,0.83,NaN,NaN,NaN,NaN,0.0,Acceptable,No Change,5.0,acceptable,No Change,NaN,NaN,NaN,NaN,NaN,15.86,99.1,NaN,16.295833,86.837514,26.0,0.05,5.0,-0.06,5.1,17.4,mixed mode


# Filter out relevant data to BCA dataset

In [9]:
# Retain overlapping columns with ASHRAE dataset
overlapping_cols = ['Mode',
                    'Indoor Temp',
                    'Indoor Humidity',
                    'Air Velocity',
                    'Globe Temperature',
                    'Outdoor Temp',
                    'Outdoor Humidity',
                    'Thermal Preference',
                    'Thermal Acceptability',
                    'Air Movement Preference']

data = data[overlapping_cols]

In [10]:
# Filter conditions related to AC and NV
data = data[data["Mode"].isin(['AC', 'NV'])].reset_index(drop=True)

# Perform imputation for missing data using MissForest

In [11]:
def impute_missing_data(data):    
    original_col_sequence = list(data.columns)
    imputer = MissForest(criterion='squared_error')
    numerical_data = data.drop(columns=['Mode',
                                        'Thermal Preference',
                                        'Thermal Acceptability',
                                        'Air Movement Preference'])
    numerical_data_cols = list(numerical_data.columns)
    imputed_numerical_data = imputer.fit_transform(numerical_data)
    imputed_numerical_data = pd.DataFrame(imputed_numerical_data, 
                                          columns=numerical_data_cols)
    imputed_data = pd.concat([data[['Mode',
                                    'Thermal Preference',
                                    'Thermal Acceptability',
                                    'Air Movement Preference']], 
                              imputed_numerical_data], 
                             axis=1)
    imputed_data = imputed_data[original_col_sequence]

    assert imputed_data.shape == data.shape
    assert imputed_data["Thermal Preference"].tolist() == data["Thermal Preference"].tolist()

    return imputed_data

data = impute_missing_data(data)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


In [12]:
print(data['Thermal Preference'].unique())
print(data['Air Movement Preference'].unique())
print(data['Thermal Acceptability'].unique())
print(data['Mode'].unique())
print(data.shape)
data.head()

['No Change' 'Warmer' 'Cooler' nan]
['More' 'No Change' 'Less' nan]
['Acceptable' 'Unacceptable' nan]
['AC' 'NV']
(79018, 10)


,Mode,Indoor Temp,Indoor Humidity,Air Velocity,Globe Temperature,Outdoor Temp,Outdoor Humidity,Thermal Preference,Thermal Acceptability,Air Movement Preference
0,AC,20.9,41.0,0.14,21.148783,12.96,84.235,No Change,Acceptable,More
1,AC,21.3,39.0,0.19,21.537667,12.96,84.235,No Change,Acceptable,No Change
2,AC,21.1,39.0,0.15,21.060083,12.96,84.235,Warmer,Unacceptable,More
3,AC,21.1,39.0,0.15,21.060083,12.96,84.235,No Change,Acceptable,No Change
4,AC,21.1,39.0,0.15,21.060083,12.96,84.235,Warmer,Acceptable,No Change


# Segment data based on different output labels and exclude rows with missing labels 

In [13]:
# thermal preference data
thermal_pref_data = data.drop(columns=['Thermal Acceptability','Air Movement Preference'])
thermal_pref_data = thermal_pref_data.dropna(subset=['Thermal Preference']).reset_index(drop=True)
assert thermal_pref_data.isnull().values.any() == False
thermal_pref_data.to_csv('data/ashrae_thermalpref_unsampled_data.csv', index=False)

In [14]:
# thermal acceptability data
thermal_acc_data = data.drop(columns=['Thermal Preference','Air Movement Preference'])
thermal_acc_data = thermal_acc_data.dropna(subset=['Thermal Acceptability']).reset_index(drop=True)
assert thermal_acc_data.isnull().values.any() == False
thermal_acc_data.to_csv('data/ashrae_thermalacc_unsampled_data.csv', index=False)

In [15]:
# air movement preference data
air_pref_data = data.drop(columns=['Thermal Acceptability','Thermal Preference'])
air_pref_data = air_pref_data.dropna(subset=['Air Movement Preference']).reset_index(drop=True)
assert air_pref_data.isnull().values.any() == False
air_pref_data.to_csv('data/ashrae_airpref_unsampled_data.csv', index=False)

# Perform resampling to address class imbalance issues

In [16]:
def rebalance_class_labels(data, cat_idx, input_idx, label_idx):
    smotenc = SMOTENC(random_state=0, categorical_features=cat_idx)
    X_res, y_res = smotenc.fit_resample(data.iloc[:, input_idx], 
                                        data.iloc[:, label_idx])
    
    return pd.concat([X_res, y_res], axis=1)

In [17]:
print('Before Resampling')
print(thermal_pref_data['Thermal Preference'].value_counts())

thermal_pref_data = rebalance_class_labels(thermal_pref_data, 
                                           [0], 
                                           list(range(thermal_pref_data.shape[1]-1)), 
                                           [-1])

print('After Resampling')
print(thermal_pref_data['Thermal Preference'].value_counts())

Before Resampling
No Change    33750
Cooler       22192
Warmer       11594
Name: Thermal Preference, dtype: int64
After Resampling
No Change    33750
Warmer       33750
Cooler       33750
Name: Thermal Preference, dtype: int64


In [18]:
print('Before Resampling')
print(thermal_acc_data['Thermal Acceptability'].value_counts())

thermal_acc_data = rebalance_class_labels(thermal_acc_data, 
                                          [0], 
                                          list(range(thermal_acc_data.shape[1]-1)), 
                                          [-1])

print('After Resampling')
print(thermal_acc_data['Thermal Acceptability'].value_counts())

Before Resampling
Acceptable      32042
Unacceptable    12645
Name: Thermal Acceptability, dtype: int64
After Resampling
Acceptable      32042
Unacceptable    32042
Name: Thermal Acceptability, dtype: int64


In [19]:
print('Before Resampling')
print(air_pref_data['Air Movement Preference'].value_counts())

air_pref_data = rebalance_class_labels(air_pref_data, 
                                       [0], 
                                       list(range(air_pref_data.shape[1]-1)), 
                                       [-1])

print('After Resampling')
print(air_pref_data['Air Movement Preference'].value_counts())

Before Resampling
No Change    13907
More          9369
Less          2107
Name: Air Movement Preference, dtype: int64
After Resampling
More         13907
No Change    13907
Less         13907
Name: Air Movement Preference, dtype: int64


# Save processed data

In [20]:
thermal_pref_data.to_csv('data/ashrae_thermalpref_sampled_data.csv', index=False)
thermal_acc_data.to_csv('data/ashrae_thermalacc_sampled_data.csv', index=False)
air_pref_data.to_csv('data/ashrae_airpref_sampled_data.csv', index=False)